<h1>거리두기 장기화에 따른 유동인구 분석<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Dataset-load,-pre-processing" data-toc-modified-id="Dataset-load,-pre-processing-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Dataset load, pre-processing</a></span><ul class="toc-item"><li><span><a href="#지하철-노선별-역별-승하차-인원-정보" data-toc-modified-id="지하철-노선별-역별-승하차-인원-정보-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>지하철 노선별 역별 승하차 인원 정보</a></span></li><li><span><a href="#S-DoT-도시데이터-센서-유동인구-측정-정보-(->-보류)" data-toc-modified-id="S-DoT-도시데이터-센서-유동인구-측정-정보-(->-보류)-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>S-DoT 도시데이터 센서 유동인구 측정 정보 (-&gt; 보류)</a></span></li><li><span><a href="#한강공원-주차장-월별-이용-현황" data-toc-modified-id="한강공원-주차장-월별-이용-현황-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>한강공원 주차장 월별 이용 현황</a></span></li></ul></li></ul></div>

In [1]:
import csv
import pandas as pd

# Dataset load, pre-processing

## 지하철 노선별 역별 승하차 인원 정보


- `날짜(idx)`, `노선명`, `역명`, `승차총승객수`, `하차총승객수`, `등록일자`, `''`의 컬럼으로 구성  
- 결측치 없음  
- `날짜`와 `하차총승객수` 컬럼을 분석에 사용  
    _(수도권 지역에서의 유입을 고려, 승차승객 보다 하차승객 중에서 서울에 실질적으로 방문할 목적의 사람들이 많을 것이라고 판단)_
- 동일 일자에 대해 여러 역에서의 `하차총승객수`가 여러 행에 걸쳐 존재  
    => 동일 일자의 데이터들을 합산하여 한 행으로 표현하는 데이터프레임으로 재구성

In [2]:
# 데이터셋 로드
subway = pd.DataFrame()

files = ['SUBWAY_202010.csv', 'SUBWAY_202011.csv', 'SUBWAY_202012.csv', 'SUBWAY_202101.csv', 'SUBWAY_202102.csv', \
         'SUBWAY_202103.csv', 'SUBWAY_202104.csv', 'SUBWAY_202105.csv', 'SUBWAY_202106.csv', 'SUBWAY_202107.csv', \
         'SUBWAY_202108.csv', 'SUBWAY_202109.csv']

for f in files:
    subway = pd.concat([subway, pd.read_csv('data/' + f)])
subway.columns = ["노선명", "역명", "승차총승객수", "하차총승객수", "등록일자", ""]
subway

,노선명,역명,승차총승객수,하차총승객수,등록일자,
20201001,2호선,시청,2118,1605,20201004,NaN
20201001,1호선,청량리(서울시립대입구),6567,6637,20201004,NaN
20201001,1호선,제기동,3450,3186,20201004,NaN
20201001,1호선,신설동,3386,3300,20201004,NaN
20201001,1호선,동대문,6187,6468,20201004,NaN
...,...,...,...,...,...,...
20210930,9호선,신논현,28562,28302,20211003,NaN
20210930,9호선2~3단계,언주,9859,10044,20211003,NaN
20210930,9호선2~3단계,선정릉,9308,10135,20211003,NaN
20210930,9호선2~3단계,삼성중앙,6389,6552,20211003,NaN


In [3]:
subway.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 218868 entries, 20201001 to 20210930
Data columns (total 6 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   노선명     218868 non-null  object 
 1   역명      218868 non-null  object 
 2   승차총승객수  218868 non-null  int64  
 3   하차총승객수  218868 non-null  int64  
 4   등록일자    218868 non-null  int64  
 5           0 non-null       float64
dtypes: float64(1), int64(3), object(2)
memory usage: 11.7+ MB


In [4]:
# 사용하지 않는 컬럼 제거
subway.drop(['노선명', '역명', '승차총승객수', '등록일자', ''], axis=1, inplace=True)
subway.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 218868 entries, 20201001 to 20210930
Data columns (total 1 columns):
 #   Column  Non-Null Count   Dtype
---  ------  --------------   -----
 0   하차총승객수  218868 non-null  int64
dtypes: int64(1)
memory usage: 3.3 MB


In [5]:
# 동일 일자의 데이터 합산하여 데이터프레임 재구성
date_s = [20201001, 20201101, 20201201, 20210101, 20210201, 20210301, 20210401, 20210501, 20210601, 20210701, 20210801, 20210901]
date_e = [20201031, 20201130, 20201231, 20210131, 20210228, 20210331, 20210430, 20210531, 20210630, 20210731, 20210831, 20210930]

df = {}
for s, e in zip(date_s, date_e):
    for d in range(s, e+1):        
        date = str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:]
        df[date] = sum(subway.loc[d, '하차총승객수'])
subway = pd.DataFrame(df.items(), columns=['날짜', '총승객수'])
subway['날짜'] = pd.to_datetime(subway['날짜'])
subway

,날짜,총승객수
0,2020-10-01,1775583
1,2020-10-02,2441986
2,2020-10-03,2993972
3,2020-10-04,2701215
4,2020-10-05,6159619
...,...,...
360,2021-09-26,3074753
361,2021-09-27,6261902
362,2021-09-28,6273005
363,2021-09-29,6053994


In [6]:
subway.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 365 entries, 0 to 364
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   날짜      365 non-null    datetime64[ns]
 1   총승객수    365 non-null    int64         
dtypes: datetime64[ns](1), int64(1)
memory usage: 5.8 KB


## S-DoT 도시데이터 센서 유동인구 측정 정보 (-> 보류)

- `기관 명`, `송신 서버 번호`, `데이터 번호`, `모델명`, `시리얼`, `서버타입`, `사이트명`, `방문자수`, `날짜`, `등록일자`의 컬럼으로 구성  
- 결측치 없음  
- `방문자수`, `날짜` 컬럼을 분석에 사용  
- 동일 일자의 방문자수가 여러행에 걸쳐 존재  
    => 동일 일자의 데이터들을 합산하여 한 행으로 표현하는 데이터프레임으로 재구성

In [28]:
# 데이터셋 로드
walk = pd.DataFrame()

files = ['2020.09.28-10.04.csv', '2020.10.05-10.11.csv', '2020.10.12-10.18.csv', '2020.10.19-10.25.csv', \
         '2020.10.26-11.01.csv', '2020.11.09-11.15.csv', '2020.11.16-11.22.csv', '2020.11.23-11.29.csv', \
         '2020.11.30-12.06.csv', '2020.12.07-12.13.csv', '2020.12.14-12.20.csv', '2020.12.21-12.27.csv', \
         '2020.12.28-01.03.csv', '2021.01.04-01.10.csv', '2021.01.11-01.17.csv', '2021.01.18-01.24.csv', \
         '2021.01.25-01.31.csv', '2021.02.01-02.07.csv', '2021.02.08-02.14.csv', '2021.02.15-02.21.csv', \
         '2021.02.22-02.28.csv', '2021.03.01-03.07.csv', '2021.03.08-03.14.csv', '2021.03.15-03.21.csv', \
         '2021.03.22-03.28.csv', '2021.03.29-04.04.csv', '2021.04.05-04.11.csv', '2021.04.12-04.18.csv']

for f in files:
    walk = pd.concat([walk, pd.read_csv("data/S-DoT_WALK_" + f, encoding='cp949')])

f = pd.read_csv('data/S-DoT_WALK_2021.04.19-04.25.csv')
f.drop(['Unnamed: 0'], axis=1, inplace=True)
walk = pd.concat([walk, f])

In [29]:
files = ['2021.04.26-05.02.csv', '2021.05.03-05.09.csv', '2021.05.10-05.16.csv', '2021.05.17-05.23.csv', \
         '2021.05.24-05.30.csv', '2021.05.31-06.06.csv', '2021.06.07-06.13.csv', '2021.06.14-06.20.csv', \
         '2021.06.21-06.27.csv', '2021.06.28-07.04.csv', '2021.07.05-07.11.csv', '2021.07.12-07.18.csv', \
         '2021.07.19-07.25.csv', '2021.07.26-08.01.csv', '2021.08.02-08.08.csv', '2021.08.09-08.15.csv', \
         '2021.08.16-08.22.csv', '2021.08.23-08.29.csv', '2021.08.30-09.05.csv', '2021.09.06-09.12.csv', \
         '2021.09.13-09.19.csv', '2021.09.20-09.26.csv', '2021.09.27-10.03.csv']

for f in files:
    walk = pd.concat([walk, pd.read_csv("data/S-DoT_WALK_" + f, encoding='cp949', low_memory = False)])
walk

,기관 명,송신 서버 번호,데이터 번호,모델명,시리얼,서버타입,사이트명,방문자수,날짜,등록일자
0,서울시,32,1,SDOT001,2992,00Original,2992,200,202010010000,2020-10-01 00:21:39
1,서울시,32,1,SDOT001,2993,00Original,2993,186,202010010000,2020-10-01 00:21:40
2,서울시,32,1,SDOT001,2994,00Original,2994,159,202010010000,2020-10-01 00:21:40
3,서울시,32,1,SDOT001,2995,00Original,2995,141,202010010000,2020-10-01 00:21:40
4,서울시,32,1,SDOT001,2996,00Original,2996,130,202010010000,2020-10-01 00:21:41
...,...,...,...,...,...,...,...,...,...,...
27248,서울시,32,1,SDOT001,4030,00Original,4030,1,202109282350,2021-09-28 23:57:23
27249,서울시,32,1,SDOT001,4007,00Original,4007,5,202109282350,2021-09-28 23:57:28
27250,서울시,32,1,SDOT001,4040,00Original,4040,14,202109282350,2021-09-28 23:57:43
27251,서울시,32,1,SDOT001,4042,00Original,4042,0,202109282350,2021-09-28 23:57:47


In [30]:
walk.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4025460 entries, 0 to 27252
Data columns (total 10 columns):
 #   Column    Dtype 
---  ------    ----- 
 0   기관 명      object
 1   송신 서버 번호  object
 2   데이터 번호    object
 3   모델명       object
 4   시리얼       object
 5   서버타입      object
 6   사이트명      object
 7   방문자수      object
 8   날짜        object
 9   등록일자      object
dtypes: object(10)
memory usage: 337.8+ MB


In [86]:
# 사용하지 않는 컬럼 제거
visited = walk.drop(['기관 명', '송신 서버 번호', '데이터 번호', '모델명', '시리얼', '서버타입', '사이트명', '등록일자'], axis=1)
visited = visited[['날짜', '방문자수']]
# visited.rename(columns={'등록일자':'날짜'},inplace=True)
visited

,날짜,방문자수
0,202010010000,200
1,202010010000,186
2,202010010000,159
3,202010010000,141
4,202010010000,130
...,...,...
27248,202109282350,1
27249,202109282350,5
27250,202109282350,14
27251,202109282350,0


In [87]:
visited.set_index('날짜',inplace=True)
visited

,방문자수
날짜,
202010010000,200
202010010000,186
202010010000,159
202010010000,141
202010010000,130
...,...
202109282350,1
202109282350,5
202109282350,14


In [85]:
# # 동일일자의 데이터 합산하여 데이터프레임 재구성
# month = ['202010', '202011', '202012', '202101', '202102', '202103', '202104', '202105', '202106', '202107', '202108', '202109']
# enddate = [31, 30, 31, 31, 28, 31, 30, 31, 30, 31, 31, 30]
# hour = ['00', '01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', \
#         '13', '14', '15', '16', '17' '18', '19' '20', '21', '22', '23']
# minute = ['00', '10', '20', '30', '40', '50']

sum(visited.loc[202109272240, '방문자수'])

9769

## 한강공원 주차장 월별 이용 현황

- `지구별`, `주차장명`, `주차대수`, `이용시간`, `날짜` 컬럼으로 구성  
- 결측치 없음  
- `날짜`, `주차대수` 컬럼을 분석에 사용  
- 동일 월의 주차대수 데이터가 여러 행에 걸쳐 존재  
    => 동일 월의 데이터들을 합산하여 한 행으로 표현하는 데이터프레임으로 재구성

In [218]:
# 데이터셋 로드
parking = pd.read_csv("data/한강공원 주차장 월별 이용 현황.csv", encoding='cp949')
parking

,지구별,주차장명,주차대수,이용시간,날짜
0,PLT-005,뚝섬4주차장,708,68086,2021/10
1,PLT-007,양화2주차장,834,69036,2021/10
2,PLT-007,양화2주차장,1019,83361,2021/10
3,PLT-003,망원23주차장,11415,1112191,2021/10
4,PLT-007,양화1주차장,3928,390477,2021/10
...,...,...,...,...,...
28491,PLT-008,여의도5주차장,2805,860432,2020/01
28492,PLT-009,반포23주차장,23590,1200177,2020/01
28493,PLT-003,망원23주차장,1382,231375,2020/01
28494,PLT-005,뚝섬1주차장,1529,379777,2020/01


In [219]:
parking.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28496 entries, 0 to 28495
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   지구별     28496 non-null  object
 1   주차장명    28496 non-null  object
 2   주차대수    28496 non-null  int64 
 3   이용시간    28496 non-null  int64 
 4   날짜      28496 non-null  object
dtypes: int64(2), object(3)
memory usage: 1.1+ MB


In [220]:
# 사용하지 않는 컬럼 제거
parking.drop(['지구별', '주차장명', '이용시간'], axis=1, inplace=True)
parking = parking[['날짜', '주차대수']]

In [221]:
parking.set_index('날짜',inplace=True)
parking.info()

<class 'pandas.core.frame.DataFrame'>
Index: 28496 entries, 2021/10 to 2020/01
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   주차대수    28496 non-null  int64
dtypes: int64(1)
memory usage: 445.2+ KB


In [222]:
# 동일 월의 데이터 합산하여 데이터프레임 재구성
date = ['2020/10', '2020/11', '2020/12', '2021/01', '2021/02', '2021/03', '2021/04', '2021/05', '2021/06', '2021/07', '2021/08', '2021/09']

df = {}
for d in date:
    df[d] = sum(parking.loc[d, '주차대수'])
parking = pd.DataFrame(df.items(), columns=['날짜', '총주차대수'])
parking['날짜'] = pd.to_datetime(parking['날짜'])
parking

,날짜,총주차대수
0,2020-10-01,38565126
1,2020-11-01,31146444
2,2020-12-01,32647266
3,2021-01-01,29960280
4,2021-02-01,33179814
5,2021-03-01,39042486
6,2021-04-01,46669716
7,2021-05-01,45453366
8,2021-06-01,48143214
9,2021-07-01,39471030
